In [1]:
from __future__ import print_function
import sys
import numpy as np
import pandas as pd
pd.options.display.max_rows = 5
import matplotlib.pyplot as plt
import gvar as gv
import fkfpi_lib as flib
print("python version:", sys.version)
print("pandas version:", pd.__version__)
print("numpy  version:", np.__version__)
print("gvar   version:", gv.__version__)

python version: 3.6.1 |Anaconda 4.4.0 (x86_64)| (default, May 11 2017, 13:04:09) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
pandas version: 0.20.1
numpy  version: 1.12.1
gvar   version: 8.3.2


In [2]:
switches = dict()
switches['ensemble'] = [
    'a15m310','a12m310','a09m310',
    'a15m220','a12m220','a09m220','a12m220S','a12m220L',
    'a15m130','a12m130']
switches['ansatz'] = dict()
switches['ansatz']['FV'] = True # True turns on NLO FV correction
switches['ansatz']['type'] = 'MA' # Type of fit: 'xpt' or 'MA'
switches['ansatz']['a2dm'] = 'avg' # avg or individual
switches['scale'] = 'PK' # PP, PK, KK, LamChi = 4 * pi * sqrt(FA * FB)

In [3]:
# Set priors for Xpt
priors = dict()
priors['L5'] = gv.gvar(0.0, 1.0)
priors['s2'] = gv.gvar(0.0, 1.0)
# Physical parameters from PDG
phys_p = dict()
# http://pdg.lbl.gov/2016/tables/rpp2016-tab-mesons-light.pdf
phys_p['mpi'] = gv.gvar(134.8, 0.3) #FLAG 2017 (16)
phys_p['mka'] = gv.gvar(494.2 , 0.3) #FLAG 2017 (16)
phys_p['F0'] = gv.gvar(80,20) #FLAG use of F0 in SU(2) correction for FK/Fpi
phys_p['meta'] = gv.gvar(547.862,0.017) #PDG
# http://pdg.lbl.gov/2015/reviews/rpp2015-rev-pseudoscalar-meson-decay-cons.pdf
phys_p['Fpi'] = gv.gvar(130.2/np.sqrt(2), 1.7/np.sqrt(2)) #PDG fpi+ eq(16)
phys_p['Fka'] = gv.gvar(155.6/np.sqrt(2), 0.4/np.sqrt(2)) #PDG fK++ eq(16)
if switches['scale'] == 'PK':
    phys_p['Lchi'] = 4*np.pi*np.sqrt(phys_p['Fpi']*phys_p['Fka'])
elif switches['scale'] == 'PP':
    phys_p['Lchi'] = 4*np.pi*phys_p['Fpi']
elif switches['scale'] == 'KK':
    phys_p['Lchi'] = 4*np.pi*phys_p['Fka']
# FLAG FK/Fpi
flag_FKFpi = gv.gvar(1.1933, 0.0029)

In [4]:
data = pd.read_csv('./github_fkfpi_v1.csv')
hisq_params = pd.read_csv('./hisq_params.csv')
mixed_params = pd.read_csv('./github_mixed_v1.csv')

In [5]:
fmt_data = flib.format_data(switches,data,mixed_params,hisq_params,priors)
#print(fmt_data)

In [6]:
print(fmt_data['p']['mju'])
print(fmt_data['p']['a2dm'])

[0.3062(13) 0.21899(92) 0.14814(37) 0.2557(19) 0.17711(85) 0.10889(26)
 0.17711(85) 0.17711(85) 0.2274(38) 0.1494(13)]
[0.03373(59) 0.01077(36) 0.00197(14) 0.03034(62) 0.01084(28) 0.001504(93)
 0.01055(30) 0.01077(30) 0.03252(82) 0.01136(32)]


In [7]:
fit = flib.fit_data(switches,fmt_data,phys_p)

In [8]:
print(fit)

Least Square Fit:
  chi2/dof [dof] = 3.7 [10]    Q = 6.2e-05    logGBF = 21.823

Parameters:
             L5   0.0002219 (83)       [     0.0 (1.0) ]  
             s2      -2e-16 +- 1       [     0.0 (1.0) ]  
          mpi 0     0.23615 (29)       [  0.23609 (29) ]  
              1     0.18863 (22)       [  0.18862 (23) ]  
              2     0.14065 (12)       [  0.14074 (12) ]  
              3     0.16544 (22)       [  0.16543 (22) ]  
              4     0.13424 (17)       [  0.13427 (17) ]  
              5    0.097904 (62)       [ 0.097896 (63) ]  
              6     0.13554 (32)       [  0.13556 (32) ]  
              7     0.13403 (15)       [  0.13402 (15) ]  
              8     0.10046 (44)       [  0.10093 (48) ]  
              9     0.08143 (23)       [  0.08128 (24) ]  
          mka 0     0.40430 (23)       [  0.40460 (25) ]  *
              1     0.32417 (19)       [  0.32416 (21) ]  
              2     0.24092 (12)       [  0.24110 (14) ]  *
              3     

In [9]:
print(flib.fkfpi_phys(phys_p,fit))

prediction from LQCD
FK / Fpi =  1.1804(13)
FK+/Fpi+ =  1.1782(13)
prediction from LQCD + meta_PDG
FK / Fpi =  1.1775(13)
FK+/Fpi+ =  1.1752(13)
None
